In [ ]:
# Install required packages in the current notebook environment
%pip install numpy tifffile

# Now import your modules
import os
import re
import numpy as np
from tifffile import imwrite, imread


In [9]:

# Set your folder path
path = '/Users/jkellerm/Library/CloudStorage/OneDrive-MichiganMedicine/0-active-projects/expansion/'
folder = os.path.join(path, 'expansion_20250604_151747/plate1a2L1')

# Regex to parse filenames
pattern = re.compile(r".*T(\d{4})F(\d{3})L(\d{2})A(\d{2})Z(\d{2})C(\d{2})\.tif$")
"""
T	Time point index (tpIndex)
F	Field of view index (fpIndex)
L	Tile or stage position index (tlIndex)
A	Additional position index (alIndex) (e.g., autofocus or acquisition round)
Z	Z-slice index (zpIndex)
C	Channel index (chIndex)
"""
# Find all files
files = [f for f in os.listdir(folder) if f.endswith('.tif')]

# Group images by T, F, L
images = {}
for fname in files:
    match = pattern.match(fname)
    if not match:
        continue
    t, f, l, a, z, c = map(int, match.groups())
    key = (t, f, l)  # group by T, F, L
    if key not in images:
        images[key] = []
    images[key].append((z, c, fname))

# Sort images within each group
for key, img_list in images.items():
    img_list.sort()  # sort by z, then c
    zs = sorted(set(z for z, c, _ in img_list))
    cs = sorted(set(c for z, c, _ in img_list))

    example_img = imread(os.path.join(folder, img_list[0][2]))
    height, width = example_img.shape
    stack = np.zeros((1, len(zs), len(cs), height, width), dtype=example_img.dtype)

    for z, c, fname in img_list:
        img = imread(os.path.join(folder, fname))
        z_index = zs.index(z)
        c_index = cs.index(c)
        stack[0, z_index, c_index, :, :] = img

    out_name = f"stack_T{key[0]}_F{key[1]}_L{key[2]}.tif"
    out_path = os.path.join(folder, out_name)
    imwrite(out_path, stack, imagej=True, metadata={'axes': 'TZCYX'})
    print(f"Saved: {out_path}")


Saved: /Users/jkellerm/Library/CloudStorage/OneDrive-MichiganMedicine/0-active-projects/expansion/expansion_20250604_151747/plate1a2L1/stack_T1_F1_L1.tif
